In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
domain = "https://vietnamnet.vn"

In [ ]:
# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get("vnnnhghhghhttps://vietnamnet.vn/thu-tuong-pham-minh-chinh-cung-thu-tuong-luxembourg-danh-trong-sam-tai-van-mieu-2139369.html")
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# import requests
# requests.get("vnnnhghhghhttps://vietnamnet.vn/thu-tuong-pham-minh-chinh-cung-thu-tuong-luxembourg-danh-trong-sam-tai-van-mieu-2139369.html").status_code
# content_element = soup.find("div", class_= "maincontent")
# arr_tag_content = []
# try: 
#   for i in content_element.find_all("p"):
#     arr_tag_content.append(i.text)
#   # if(len(contents) != url_news.index(link) + 1):
#   #   print(link)
#   #   print(content_string)
# except Exception as e:
#   print("fail to add content" + str(e))

# content_string = " ".join(arr_tag_content)
# driver.quit()

In [ ]:
def get_urls_in_pages(driver, page_urls):
  url_news = []
  for page_url in page_urls:
    driver.get(page_url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tag_elements = soup.find_all('h3', class_= 'horizontalPost__main-title')
    a_elements = [tag.find("a") for tag in tag_elements]
    for a in a_elements:
      try:
        if domain in a['href']:
          url_news.append(a['href'])
        else:
          rl_news.append(domain + a['href'])
      except:
        pass
  return url_news

def get_content_and_label_from_link(driver, url_news, label):
  contents = []
  labels = []
  for link in url_news:
    driver.get(link)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
     
    content_element = soup.find("div", class_= "maincontent")
    arr_tag_content = []
    try: 
      for i in content_element.find_all("p"):
        arr_tag_content.append(i.text)
    except Exception as e: 
      print("content faild")
      print(e)
      print(link)
    content_string = " ".join(arr_tag_content)
    contents.append(content_string)
    labels.append(label)
    if(len(contents) != url_news.index(link) + 1):
      break
    
  return contents, labels

In [ ]:
pages = []
for index in range(1, 50):
  uri = "/thoi-su/chinh-tri-page"
  pages.append(domain + uri + str(index))
driver = webdriver.Chrome(ChromeDriverManager().install())
url_title = get_urls_in_pages(driver, pages)

In [ ]:
url_title

In [ ]:
def crawl(uri, label):
  # uri = "/thoi-su/chinh-tri-page"
  pages = []
  for index in range(1, 3):
    pages.append(domain + uri + str(index))
  driver = webdriver.Chrome(ChromeDriverManager().install())
  url_title = get_urls_in_pages(driver, pages)
  time.sleep(5)
  contents, labels = get_content_and_label_from_link(driver, url_title, label)
  driver.quit()
  print(len(url_title), len(contents), len(labels))
  data = pd.DataFrame({"Link": url_title, "Contents": contents, "Labels": labels})
  data.to_csv("data_v1_"+label+".csv", encoding="utf-8")

In [ ]:
crawl("/thoi-su/chinh-tri-page", "politics")

In [ ]:
pd.read_csv("data_v1_politics.csv").head()

In [ ]:
# List of URLs to be processed
urls = ['/thoi-su/chinh-tri-page', '/kinh-doanh-page', '/the-thao-page', '/suc-khoe-page', '/giao-duc-page', '/giai-tri-page']
labels = ['politics', 'business', 'sport', 'health', 'entertainment', 'education']
# Create a ThreadPoolExecutor with 3 worker threads
with ThreadPoolExecutor(max_workers=6) as executor:
    # Submit a task for each URL in the list
    futures = [executor.submit(crawl, url, labels) for url,label in zip(urls, labels)]

    # Wait for all tasks to complete and print the results
    for i in futures:
        print(i)

In [ ]:
import threading

def get_title(url):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    title = driver.title
    driver.quit()
    return title

# create a new thread and start it
thread = threading.Thread(target=get_title, args=("Hello", "World"))
thread.start()

# wait for the thread to finish
thread.join()